# Matrix product operators (MPOs) for representing quantum Hamiltonians

In [ ]:
import numpy as np
from scipy import sparse

## Construct MPOs

In [ ]:
def construct_ising_hamiltonian_mpo(J, g, L, pbc=False):
    """
    Construct Ising Hamiltonian on a 1D lattice with `L` sites as MPO,
    for interaction parameter `J` and external field parameter `g`.
    """
    # TODO: implement this function (you can ignore the case pbc=True for now)

In [ ]:
Alist_ising = construct_ising_hamiltonian_mpo(1.1, 0.7, 5)

In [ ]:
# check dimensions (should return True)
Alist_ising[0].shape == (2, 2, 1, 3)

In [ ]:
# check dimensions (should return True)
Alist_ising[1].shape == (2, 2, 3, 3)

In [ ]:
# check dimensions (should return True)
Alist_ising[-1].shape == (2, 2, 3, 1)

In [ ]:
# example
Alist_ising[1]

In [ ]:
def construct_cluster_hamiltonian_mpo(J, L):
    """
    Construct the cluster state Hamiltonian as MPO.
    """
    # TODO: implement this function

In [ ]:
Alist_cluster = construct_cluster_hamiltonian_mpo(0.9, 5)

In [ ]:
# example: show dimensions
Alist_cluster[0].shape

In [ ]:
# example: show dimensions
Alist_cluster[1].shape

## Utility functions

In [ ]:
def mpo_to_full_tensor(Alist):
    """
    Construct the full tensor corresponding to the MPO tensors `Alist`.

    The i-th MPO tensor Alist[i] is expected to have dimensions (m[i], n[i], D[i], D[i+1]),
    with `m` and `n` the list of logical dimensions and `D` the list of virtual bond dimensions.
    
    The returned tensor has dimensions m[0] x ... x m[L-1] x n[0] x ... x n[L-1]

    Note: Should only be used for debugging and testing.
    """
    # consistency check
    assert Alist[0].ndim == 4
    # use leftmost virtual bond as first dimension
    T = np.transpose(Alist[0], (2, 0, 1, 3))
    # contract virtual bonds
    for i in range(1, len(Alist)):
        T = np.tensordot(T, Alist[i], axes=(-1, 2))
    # contract leftmost and rightmost virtual bond (has no influence if these virtual bond dimensions are 1)
    assert T.shape[0] == T.shape[-1]
    T = np.trace(T, axis1=0, axis2=-1)
    # now T has dimensions m[0] x n[0] x m[1] x n[1] ... m[d-1] x n[d-1];
    # as last step, we group the `m` dimensions together, and likewise the `n` dimensions
    T = np.transpose(T, list(range(0, T.ndim, 2)) + list(range(1, T.ndim, 2)))
    return T

In [ ]:
# example
mpo_to_full_tensor([np.random.randn(3, 4, 1, 5), np.random.randn(7, 2, 5, 3), np.random.randn(6, 5, 3, 1)]).shape

## Construct quantum Hamiltonian as sparse matrix (as reference)

### Transverse-field Ising Hamiltonian

In [ ]:
def adjacency_1D_lattice(L, pbc=True):
    """
    Construct the adjacency matrix for a 1D lattice with `L` sites.
    The optional parameter `pbc` specifies whether periodic boundary conditions
    should be used.
    """
    assert L > 1
    # special case
    if L == 2:
        return np.array([[0, 1], [1, 0]])
    if pbc:
        # periodic boundary conditions
        return np.roll(np.identity(L, dtype=int), -1, axis=0) + np.roll(np.identity(L, dtype=int), 1, axis=0)
    else:
        # open boundary conditions
        return np.diag(np.ones(L - 1, dtype=int), k=-1) + np.diag(np.ones(L - 1, dtype=int), k=1)

In [ ]:
# should be symmetric
np.linalg.norm(adjacency_1D_lattice(7) - adjacency_1D_lattice(7).T)

In [ ]:
# each site should have 2 neighbors (for periodic boundary conditions)
np.sum(adjacency_1D_lattice(7), axis=0)

In [ ]:
# example
adjacency_1D_lattice(5, pbc=False)

In [ ]:
# Note: this is a solution of Exercise 9.2 (b)
def construct_ising_hamiltonian_sparse(J, g, adj):
    """
    Construct Ising Hamiltonian as sparse matrix,
    for interaction parameter `J` and external field parameter `g`.
    `adj` is the adjacency matrix of the underlying lattice.
    """
    # Pauli-X and Z matrices
    X = sparse.csr_matrix([[0., 1.], [1.,  0.]])
    Z = sparse.csr_matrix([[1., 0.], [0., -1.]])
    # overall number of lattice sites
    L = adj.shape[0]
    H = sparse.csr_matrix((2**L, 2**L), dtype=float)
    for j in range(L):
        for k in range(j+1, L):
            if adj[j, k] > 0:
                H -= J * sparse.kron(sparse.eye(2**j),
                         sparse.kron(Z,
                         sparse.kron(sparse.eye(2**(k-j-1)),
                         sparse.kron(Z,
                                     sparse.eye(2**(L-k-1))))))
    # external field
    for j in range(L):
        H -= g * sparse.kron(sparse.eye(2**j), sparse.kron(X, sparse.eye(2**(L-j-1))))
    return H

In [ ]:
adj = adjacency_1D_lattice(5, pbc=False)
Hising = construct_ising_hamiltonian_sparse(1.1, 0.7, adj)
Hising

In [ ]:
# convert to NumPy array to show entries
Hising.toarray()

In [ ]:
# compare (difference should be zero)
np.linalg.norm(Hising.toarray() - np.reshape(mpo_to_full_tensor(Alist_ising), (32, 32)))

In [ ]:
# periodic boundary conditions
adj = adjacency_1D_lattice(5, pbc=True)
Hising_per = construct_ising_hamiltonian_sparse(1.1, 0.7, adj)
Hising_per

In [ ]:
# compare (difference should be zero) - this is only relevant for part (c)
np.linalg.norm(Hising_per.toarray() - np.reshape(mpo_to_full_tensor(construct_ising_hamiltonian_mpo(1.1, 0.7, 5, pbc=True)), (32, 32)))

### Cluster state Hamiltonian

In [ ]:
def construct_cluster_hamiltonian_sparse(J, L):
    """
    Construct the cluster state Hamiltonian as sparse matrix
    on a one-dimensional lattice with open boundary conditions.
    """
    # Pauli-X and Z matrices
    X = sparse.csr_matrix([[0., 1.], [1.,  0.]])
    Z = sparse.csr_matrix([[1., 0.], [0., -1.]])
    H = sparse.csr_matrix((2**L, 2**L), dtype=float)
    h = sparse.kron(sparse.kron(Z, X), Z)
    for j in range(L-2):
        H -= sparse.kron(sparse.eye(2**j),
             sparse.kron(h,
                         sparse.eye(2**(L-j-3))))
    return J*H

In [ ]:
Hcluster = construct_cluster_hamiltonian_sparse(0.9, 5)

In [ ]:
# compare (difference should be zero)
np.linalg.norm(Hcluster.toarray() - np.reshape(mpo_to_full_tensor(Alist_cluster), (32, 32)))